# Checkpoint 7 - Challenge
Now take your Keras skills and go build another neural network. Pick your data set, but it should be one of abstract types, possibly even nonnumeric, and use Keras to make five implementations of your network. Compare them both in computational complexity as well as in accuracy and given that tradeoff decide which one you like best.

Your dataset should be sufficiently large for a neural network to perform well (samples should really be in the thousands here) and try to pick something that takes advantage of neural networks’ ability to have both feature extraction and supervised capabilities, so don’t pick something with an easy to consume list of features already generated for you (though neural networks can still be useful in those contexts).

Note that if you want to use an unprocessed image dataset, scikit-image is a useful package for converting to importable numerics.

In [6]:
import os
os.chdir('C:\\Users\\M246047\\Documents\\Python')
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta
import re
import tensorflow
import keras

from sklearn.datasets import fetch_openml
from keras.datasets import mnist
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import scipy.stats as stats

# Import various componenets for model building
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

In [4]:
mnist = fetch_openml('Fashion-MNIST', version=1, cache=True)

indices = np.random.choice(70000,10000)
X = mnist.data[indices] / 255.0
y = mnist.target[indices]

print(X.shape, y.shape)

(10000, 784) (10000,)


In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


# Model One - Multilayer Perceptron 

Test loss: 0.08862657889742404 <br>
Test accuracy: 0.9746000170707703

We're starting out with a very good model - I'm curious to see if others can do better.

In [9]:
# Change shape 
# Note that our images are 28*28 pixels, so in reshaping to arrays we want
# 60,000 arrays of length 784, one for each image
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# Convert to float32 for type consistency
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize values to 1 from 0 to 255 (256 values of pixels)
x_train /= 255
x_test /= 255

# Print sample sizes
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

60000 train samples
10000 test samples


In [10]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(10, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.4336 - accuracy: 0.8733 - val_loss: 0.1890 - val_accuracy: 0.9435
Epoch 2/10
60000/60000 [==============================] - 2s 39us/step - loss: 0.2053 - accuracy: 0.9390 - val_loss: 0.1387 - val_accuracy: 0.9585
Epoch 3/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.1610 - accuracy: 0.9516 - val_loss: 0.1259 - val_accuracy: 0.9623
Epoch 4/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.1365 - accuracy: 0.9583 - val_loss: 0.1099 - val_accuracy: 0.9659
Epoch 5/10
60000/60000 [==============================] - 2s 41us/step - loss: 0.1192 - accuracy: 0.9637 - val_loss: 0.0992 - val_accuracy: 0.9707
Epoch 6/10
60000/60000 [==============================] - 2s 40us/step - loss: 0.1075 - accuracy: 0.9672 - val_loss: 0.1016 - val_accuracy: 0.9702
Epoch 7/10
60000/60000 [==============================] - 2s 39us/st

# Model 2 - CNN

Test loss: 0.02581354253039426 <br>
Test accuracy: 0.9922999739646912

This mode performs almost perfectly.

In [12]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 119s 2ms/step - loss: 0.2816 - accuracy: 0.9129 - val_loss: 0.0608 - val_accuracy: 0.9802
Epoch 2/10
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0915 - accuracy: 0.9736 - val_loss: 0.0416 - val_accuracy: 0.9858
Epoch 3/10
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0683 - accuracy: 0.9809 - val_loss: 0.0325 - val_accuracy: 0.9889
Epoch 4/10
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0560 - accuracy: 0.9836 - val_loss: 0.0308 - val_accuracy: 0.9899
Epoch 5/10
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0475 - accuracy: 0.9861 - val_loss: 0.0319 - val_accuracy: 0.9899
Epoch 6/10
60000/60000 [==============================] - 119s 2ms/step - loss: 0.0429 - accuracy: 0.9869 - val_loss: 0.0287 - val_accura

# Model 3 - CNN with Fewer Layers and Tanh Activation

Test loss: 0.03989340436264174 <br>
Test accuracy: 0.9876999855041504

This model performs well, but not as well as the model with more layers.

In [17]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 100s 2ms/step - loss: 0.2307 - accuracy: 0.9300 - val_loss: 0.0924 - val_accuracy: 0.9725
Epoch 2/10
60000/60000 [==============================] - 101s 2ms/step - loss: 0.0863 - accuracy: 0.9747 - val_loss: 0.0617 - val_accuracy: 0.9799
Epoch 3/10
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0661 - accuracy: 0.9799 - val_loss: 0.0508 - val_accuracy: 0.9832
Epoch 4/10
60000/60000 [==============================] - 103s 2ms/step - loss: 0.0539 - accuracy: 0.9836 - val_loss: 0.0509 - val_accuracy: 0.9831
Epoch 5/10
60000/60000 [==============================] - 103s 2ms/step - loss: 0.0476 - accuracy: 0.9854 - val_loss: 0.0548 - val_accuracy: 0.9839
Epoch 6/10
60000/60000 [==============================] - 104s 2ms/step - loss: 0.0416 - accuracy: 0.9872 - val_loss: 0.0523 - val_accura

# Model 4 - CNN with More Layers and Relu Activation

Test loss: 0.05307199218791486 <br>
Test accuracy: 0.9884999990463257

This model performs well, but not as well as the model with more layers.

In [16]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 85s 1ms/step - loss: 0.6255 - accuracy: 0.8009 - val_loss: 0.0937 - val_accuracy: 0.9755
Epoch 2/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.2408 - accuracy: 0.9363 - val_loss: 0.0691 - val_accuracy: 0.9839
Epoch 3/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.1836 - accuracy: 0.9525 - val_loss: 0.0737 - val_accuracy: 0.9833
Epoch 4/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.1639 - accuracy: 0.9579 - val_loss: 0.0537 - val_accuracy: 0.9871
Epoch 5/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.1493 - accuracy: 0.9618 - val_loss: 0.0546 - val_accuracy: 0.9862
Epoch 6/10
60000/60000 [==============================] - 88s 1ms/step - loss: 0.1399 - accuracy: 0.9658 - val_loss: 0.0623 - val_accuracy: 0.

# Model 5 - Hierarchical RNN with Three Epochs and 64 Batch Size

Test loss: 0.17047725892663002 <br>
Test accuracy: 0.9505000114440918

In [22]:
# Training parameters.
batch_size = 64
num_classes = 10
epochs = 3

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 185s 3ms/step - loss: 0.8975 - accuracy: 0.6965 - val_loss: 0.5738 - val_accuracy: 0.8064
Epoch 2/3
60000/60000 [==============================] - 185s 3ms/step - loss: 0.3634 - accuracy: 0.8860 - val_loss: 0.2406 - val_accuracy: 0.9251
Epoch 3/3
60000/60000 [==============================] - 186s 3ms/step - loss: 0.2138 - accuracy: 0.9356 - val_loss: 0.1654 - val_accuracy: 0.9500
Test loss: 0.1654222881436348
Test accuracy: 0.949999988079071


# Model 6 - Hierarchical RNN with Six Epochs and 32 Batch Size

Test loss: 0.07090047200620174 <br>
Test accuracy: 0.9779999852180481

In [21]:
# Training parameters.
batch_size = 32
num_classes = 10
epochs = 6

# Embedding dimensions.
row_hidden = 32
col_hidden = 32

# The data, shuffled and split between train and test sets.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshapes data to 4D for Hierarchical RNN.
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Converts class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

row, col, pixel = x_train.shape[1:]

# 4D input.
x = Input(shape=(row, col, pixel))

# Encodes a row of pixels using TimeDistributed Wrapper.
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)

# Encodes columns of encoded rows.
encoded_columns = LSTM(col_hidden)(encoded_rows)

# Final predictions and model.
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Training.
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

# Evaluation.
scores = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/6
60000/60000 [==============================] - 207s 3ms/step - loss: 0.7853 - accuracy: 0.7275 - val_loss: 0.3489 - val_accuracy: 0.8891
Epoch 2/6
60000/60000 [==============================] - 205s 3ms/step - loss: 0.2721 - accuracy: 0.9169 - val_loss: 0.1645 - val_accuracy: 0.9490
Epoch 3/6
60000/60000 [==============================] - 198s 3ms/step - loss: 0.1765 - accuracy: 0.9463 - val_loss: 0.1329 - val_accuracy: 0.9580
Epoch 4/6
60000/60000 [==============================] - 194s 3ms/step - loss: 0.1332 - accuracy: 0.9599 - val_loss: 0.1017 - val_accuracy: 0.9693
Epoch 5/6
60000/60000 [==============================] - 199s 3ms/step - loss: 0.1092 - accuracy: 0.9668 - val_loss: 0.0971 - val_accuracy: 0.9694
Epoch 6/6
60000/60000 [==============================] - 194s 3ms/step - loss: 0.0934 - accuracy: 0.9712 - val_loss: 0.0709 - val_accuracy: 0.

| Model | Test Loss | Test Accuracy | 
|------|------|------|
| Multilayer Perceptron with Five Layers | 0.0886 | 0.9746 |
| CNN with Eight Layers and Relu Activation | 0.0258 | 0.9923 |
| CNN with Six Layers and Tanh Activation | 0.0528 | 0.9857 |
| CNN with Twelve Layers and Relu Activation | 0.0531 | 0.9885 |
| Hierarchical RNN Three Epochs and 64 Batch Size |  0.1654 | 0.9500 |
| Hierarchical RNN with Six Epochs and 32 Batch Size | 0.07090 | 0.9780 |


The CNN model with Relu activation and eight layers performed best with the lowest test lost and highest accuracy. I'm curious that the same model with twelve layers didn't perform better - perhaps this is because of overfitting?

The Hierarchical RNN model improved with a larger number of epochs and smaller batch size. My guess is that the batch size didn't matter as much and the improvement is due to the increase in epochs.